## Reference Resources
[all-pytorch-loss-function](https://analyticsindiamag.com/all-pytorch-loss-function/)

In [10]:
import torch
import torch.nn as nn
import numpy as np

## CrossEntroypLoss

In [30]:
import torch

input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input, input.shape)
print(target, target.shape)

tensor([[ 1.8719, -1.0069,  0.4193,  0.0851,  0.3153],
        [-1.4044,  1.0306, -0.3020, -0.3410,  0.4111],
        [ 0.2907, -0.6874, -0.5904, -1.2338, -1.3248]], requires_grad=True) torch.Size([3, 5])
tensor([1, 3, 4]) torch.Size([3])


In [52]:
from mmdet.models.losses import CrossEntropyLoss

import torch
from torch import nn
import torch.nn.functional as F

pred_score = torch.tensor([[13., 3., 2., 5.],
                           [1., 8., 20., 2.]])
target = torch.tensor([2, 0])

class_weight = [0.1, 0.2, 0.2, 0.4]
class_weight = torch.tensor(class_weight)

def method1():
    """https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html
    """
    loss = F.cross_entropy(pred_score, target, ignore_index=255)
    print(loss)


def method2():
    print("=========method2=========")
    print(torch.sum(class_weight))
    loss = nn.CrossEntropyLoss(reduction='none')
    loss_ce = loss(pred_score, target)
    print(loss_ce)

    loss = nn.CrossEntropyLoss(weight=class_weight, reduction='none')
    loss_ce = loss(pred_score, target)
    print(loss_ce)

    loss = nn.CrossEntropyLoss(weight=class_weight, reduction='mean')
    loss_ce = loss(pred_score, target)
    print(loss_ce, 'mean')

    loss = nn.CrossEntropyLoss(weight=class_weight, reduction='sum')
    loss_ce = loss(pred_score, target)
    print(loss_ce)
    print("=========method2=========")

def method3():
    pred_score = torch.tensor([[13., 3., 2., 5.],
                           [1., 8., 20., 2.]])
    target = torch.tensor([2, 0])

    class_weight = [0.1, 0.2, 0.2, 0.4]
    class_weight = torch.tensor(class_weight)
    
    loss = CrossEntropyLoss(use_sigmoid=False, 
                            class_weight=class_weight, 
                            reduction='mean', 
                            loss_weight=1.0)
    mask = torch.tensor([0, 1])
    loss_ce = loss(pred_score, target, mask, avg_factor=1)
    print(loss_ce)

    mask = torch.tensor([False, True])
    pred_score = pred_score[mask]
    target = target[mask]
    loss_ce = loss(pred_score, target)
    print(loss_ce)


# pred_score = torch.tensor([[13., 3., 2., 5., 1.],
#                            [1., 8., 20., 2., 3.],
#                            [1., 14., 3., 5., 3.]])
# print(pred_score)
# pred_score_soft = F.softmax(pred_score, dim=1)
# print(pred_score_soft)
# pred_score_soft_log = pred_score_soft.log()

# target = torch.tensor([3, 1, 255])
# loss = F.cross_entropy(pred_score, target, ignore_index=255)
# print(loss)

method1()
method2()
method3()

tensor(15.0002)
=========method2=========
tensor(0.9000)
tensor([11.0004, 19.0000])
tensor([2.2001, 1.9000])
tensor(13.6669) mean
tensor(4.1001)
=========method2=========
tensor(1.9000)
tensor(1.9000)


## KL loss

In [3]:
import torch
import torch.nn.functional as F

pred = torch.randn(4, 200, 200, 16, 18)
target = torch.randn(4, 200, 200, 16, 18)

pred = torch.randn(4, 18)
target = torch.randn(4, 18)
T = 10


kd_loss = F.kl_div(
        F.log_softmax(pred / T, dim=1), target, reduction='none').mean(1) * (
            T * T)
print(kd_loss.shape)

torch.Size([4])


In [2]:
import torch
from mmdet.models.losses import KnowledgeDistillationKLDivLoss, FocalLoss
from mmdet.models.losses import CrossEntropyLoss

def test_kl_loss():
    kl_loss = KnowledgeDistillationKLDivLoss()

    pred = torch.randn(4, 18)
    target = torch.randn(4, 18)
    weight = torch.randn(4)
    print(weight.shape)
    dd = pred.reshape(2, -1)
    print(dd.shape)

    loss = kl_loss(pred, target, weight)
    print(loss)

def test_focal_loss():
    focal_loss = FocalLoss()

    pred = torch.randn(4, 18)
    target = torch.randint(0, 15, (4,))
    print(pred.shape, target.shape)

    loss = focal_loss(pred, target)
    print(loss)

test_focal_loss()

torch.Size([4, 18]) torch.Size([4])
tensor(0.2225)


## Focal Loss

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from mmdet.models.losses import FocalLoss

pred = torch.tensor([[0.1, 0.2, 0.3, -0.5, 0.6],
                     [0.2, 0.3, 0.4, -0.5, 0.6]])
target = torch.tensor([0, 4])
print(pred.shape, target.shape)
num_classes = pred.size(1)
focal_loss = FocalLoss()

num_pos_occ = torch.sum(target < 4)
occ_avg_factor = num_pos_occ * 1.0
print(occ_avg_factor)
loss = focal_loss(pred, target, avg_factor=occ_avg_factor)

# target = F.one_hot(target, num_classes=num_classes + 1)
# target = target[:, :num_classes]
# print(target)
# loss = F.binary_cross_entropy_with_logits(
#     pred, target, reduction='none')
print(loss)

torch.Size([2, 5]) torch.Size([2])
tensor(1.)
tensor(1.5061)


## L1 loss

In [8]:
import torch
import torch.nn.functional as F

data1 = torch.arange(12).reshape(2, 3, 2).to(torch.float32)
data2 = torch.arange(12, 0, -1).reshape(2, 3, 2).to(torch.float32)
print(data1)
print(data2, data2.shape)

loss_none = F.l1_loss(data1, data2, reduction='none')
print(loss_none, loss_none.shape, "loss")

loss = F.l1_loss(data1, data2, reduction='mean')
print(loss, "loss mean")

loss = loss_none.sum() / data1.numel()
print(loss, "loss")


tensor([[[ 0.,  1.],
         [ 2.,  3.],
         [ 4.,  5.]],

        [[ 6.,  7.],
         [ 8.,  9.],
         [10., 11.]]])
tensor([[[12., 11.],
         [10.,  9.],
         [ 8.,  7.]],

        [[ 6.,  5.],
         [ 4.,  3.],
         [ 2.,  1.]]]) torch.Size([2, 3, 2])
tensor([[[12., 10.],
         [ 8.,  6.],
         [ 4.,  2.]],

        [[ 0.,  2.],
         [ 4.,  6.],
         [ 8., 10.]]]) torch.Size([2, 3, 2]) loss
tensor(6.) loss mean
tensor(6.) loss


In [13]:
import torch
import torch.nn.functional as F

data1 = torch.arange(12).reshape(2, 3, 2).to(torch.float32)
data2 = torch.arange(12, 0, -1).reshape(2, 3, 2).to(torch.float32)
print(data1)
print(data2, data2.shape)

data11 = data1[0]
data21 = data2[0]

data12 = data1[1]
data22 = data2[1]

loss1 = F.l1_loss(data11, data21, reduction='mean')
loss2 = F.l1_loss(data12, data22, reduction='mean')
print(loss1, loss2)

loss = torch.stack([loss1, loss2]).mean()
print(loss)

tensor([[[ 0.,  1.],
         [ 2.,  3.],
         [ 4.,  5.]],

        [[ 6.,  7.],
         [ 8.,  9.],
         [10., 11.]]])
tensor([[[12., 11.],
         [10.,  9.],
         [ 8.,  7.]],

        [[ 6.,  5.],
         [ 4.,  3.],
         [ 2.,  1.]]]) torch.Size([2, 3, 2])
tensor(7.) tensor(5.)
tensor(6.)


In [14]:
import torch
import torch.nn.functional as F


### https://pytorch.org/docs/stable/generated/torch.nn.L1Loss.html
def cal_loss(input, target):
    """
    MAE计算细节
    """
    print(target - input)
    print(torch.abs(target - input))
    ### Calculate the absolute
    diff = input - target
    abs_diff = torch.abs(diff)
    mae = torch.mean(abs_diff)
    print(mae)


def calc_loss_with_different_methods(pred, tgt):
    print("calc_loss_with_different_methods")
    ## Method 1
    l1_loss = nn.L1Loss(reduction='none')
    output = l1_loss(pred, tgt)
    print(output, output.shape)
    
    output = torch.mean(output) # the same with nn.L1Loss(reduction='mean')
    print(output)

    ## Method 2
    l1_loss = nn.L1Loss()
    output = l1_loss(pred, tgt)
    print(output)

    ## Method 3
    output = F.l1_loss(pred, tgt)
    print(output)


pred = torch.arange(16).reshape(2, 4, 2).to(torch.float32)
target = torch.arange(16, 0, step=-1).reshape(2, 4, 2).to(torch.float32)

calc_loss_with_different_methods(pred, target)


# mae_loss = nn.L1Loss(reduction='mean')
# input = torch.randn(3, 5)
# target = torch.randn(3, 5)
# target = torch.tensor([[1,2,3,4,5],[1,2,3,4,5], [1,2,3,4,5.0]])
# target = torch.tensor([[1,2,3,4,5.0]])

# print(input, target)
# cal_loss(input, target)

# output = mae_loss(input, target)
# print(output, output.shape)

# target = torch.tensor([[1,2,3,4,5],[1,2,3,4,5], [1,2,3,4,5.0]])
# output = mae_loss(input, target)
# print(output, output.shape)




calc_loss_with_different_methods
tensor([[[16., 14.],
         [12., 10.],
         [ 8.,  6.],
         [ 4.,  2.]],

        [[ 0.,  2.],
         [ 4.,  6.],
         [ 8., 10.],
         [12., 14.]]]) torch.Size([2, 4, 2])
tensor(8.)
tensor(8.)
tensor(8.)


In [30]:
import torch.nn.functional as F


def pixel_wise_loss(pred, target, mask=None, weight=1.0):
    l1_dist = F.l1_loss(pred, target, reduction="none")

    if mask is None:
        return torch.mean(l1_dist)
    
    loss1 = weight * l1_dist * mask
    loss2 = l1_dist * (1 - mask)
    
    loss = torch.mean(loss1 + loss2)
    return loss

pred = torch.arange(0, 9).reshape(3, 3).to(torch.float32)

# target = torch.arange(9, 0, -1).reshape(3, 3).to(torch.float32)
target = torch.arange(1, 10).reshape(3, 3).to(torch.float32)
# target[0, 0] += 2

target[2, 2] += 2


print(pred)
print(target)

mask = torch.zeros_like(target)
mask[:2, :2] = 1.
print(mask)

loss = pixel_wise_loss(pred, target)
# loss = pixel_wise_loss(pred, target, mask=mask, weight=2.0)

print(loss)




tensor(1.2222)
tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])
tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8., 11.]])
tensor([[1., 1., 0.],
        [1., 1., 0.],
        [0., 0., 0.]])
tensor(1.2222)


## Smooth L1Loss

## MSE Loss

In [6]:
import torch

mse_criterion = torch.nn.MSELoss(reduction='sum')

pred = torch.tensor([[0, 2, 2, 4]], dtype=torch.float32)
target = torch.tensor([[2, 3, 4, 5]], dtype=torch.float32)

mse_loss = mse_criterion(pred, target)
print(mse_loss)


tensor(10.)


In [ ]:

### Pytorch实现
input = torch.randn(3, 4, requires_grad=True)
target = torch.randn(3, 4)
mse_loss = nn.MSELoss()
output = mse_loss(input, target)
print(output)
output.backward()
print('input -: ', input)
print('target -: ', target)
print('output -: ', output)

pred = torch.arange(0, 9).reshape(3, 3).to(torch.float32) / 9.0
# pred[0, 0] = 0.5
# pred[2, 2] = 0.1




target = torch.arange(9, 0, -1).reshape(3, 3).to(torch.float32) / 9.0
print(target)

pred[2] = target[2]
# pred[2, :2] = target[2, :2]
print(pred)

class WeightedLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse_loss = nn.MSELoss()
    
    def forward(self, pred, target):
        weight = -torch.log(1 - target + 1e-5)
        weighted_diff = torch.multiply((pred - target), weight)
        
        return torch.sum(torch.pow(weighted_diff, 2))

mse_loss = WeightedLoss()
output = mse_loss(pred, target)
print(output)

## Binary Cross Entropy(nn.BCELoss)
[https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html)

[https://gombru.github.io/2018/05/23/cross_entropy_loss/](https://gombru.github.io/2018/05/23/cross_entropy_loss/)

In [56]:
### Python实现
y_pred = np.array([0.1580, 0.4137, 0.2285])
y_true = np.array([0.0, 1.0, 0.0]) #2 labels: (0,1)
def BCE(y_pred, y_true):
    total_bce_loss = np.sum(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))
    # Getting the mean BCE loss
    num_of_samples = y_pred.shape[0]
    mean_bce_loss = total_bce_loss / num_of_samples
    return mean_bce_loss
bce_value = BCE(y_pred, y_true)
print ("BCE error is: " + str(bce_value))

BCE error is: 0.43800269247783435


In [58]:
## Using Pytorch implementation
bce_loss = torch.nn.BCELoss()
sigmoid = torch.nn.Sigmoid() # Ensuring inputs are between 0 and 1
input = torch.tensor(y_pred)
target = torch.tensor(y_true)
print(input.shape, target.shape)
output = bce_loss(input, target)
print(output)

torch.Size([3]) torch.Size([3])
tensor(0.4380, dtype=torch.float64)


## BCEWithLogitsLoss(nn.BCEWithLogitsLoss)

In [25]:
from mmdet.models.losses import CrossEntropyLoss
import torch.nn.functional as F

bce_loss = CrossEntropyLoss(use_sigmoid=True)

torch.random.manual_seed(0)
input = torch.randn((3, 1))
target = torch.empty((3, 1)).random_(3)
print(input, target, target.dtype)

loss = bce_loss(input, target)
print(loss)

loss = F.binary_cross_entropy_with_logits(input, target)
print(loss)

tensor([[ 1.5410],
        [-0.2934],
        [-2.1788]]) tensor([[1.],
        [0.],
        [2.]]) torch.float32
tensor(1.7387)
tensor(1.7387)


In [14]:
import torch
import torch.nn as nn

loss = nn.BCEWithLogitsLoss(weight=torch.tensor([1.0, 1.0, 1.0]))
input = torch.randn(3, 1, requires_grad=True)
print(input.shape)

target = torch.empty((3, 1)).random_(2)
output = loss(input, target)
print(input.shape, target.shape)
print(input)
print(target)

print(output)

torch.Size([3, 1])
torch.Size([3, 1]) torch.Size([3, 1])
tensor([[-1.2800],
        [ 0.0282],
        [ 0.9930]], requires_grad=True)
tensor([[0.],
        [1.],
        [1.]])
tensor(0.4132, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [2]:
import torch
import torch.nn as nn

def example1():
    model = nn.Linear(10, 1)
    criterion = nn.BCEWithLogitsLoss()
    criterion = nn.BCELoss()

    x = torch.randn(16, 10)
    y = torch.empty(16).random_(2)  # (16, )
    print(y, y.shape)

    out = model(x)  # (16, 1)
    out = out.squeeze(dim=-1)  # (16, )

    print(out)

    loss = criterion(y, y)
    print(loss)

def example2():
    torch.manual_seed(100)
    
    criterion = nn.BCEWithLogitsLoss()
    criterion2 = nn.BCELoss()

    pred = torch.rand(16,1) * 100
    print(pred)

    sigmoid = torch.nn.Sigmoid()
    print(sigmoid(pred))

    target = torch.zeros_like(pred)
    print(target.shape)

    loss = criterion(target, pred)
    print(loss)

    print(criterion2(target, sigmoid(pred)))

example2()

tensor([[11.1664],
        [81.5843],
        [26.2562],
        [48.3878],
        [67.6504],
        [75.3911],
        [26.2691],
        [ 4.2840],
        [20.8029],
        [11.8037],
        [12.1695],
        [73.5599],
        [71.1765],
        [78.7581],
        [41.8305],
        [90.1415]])
tensor([[1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [0.9864],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000]])
torch.Size([16, 1])
tensor(0.6931)
tensor(27.6075)


## CosineEmbeddingLoss
https://pytorch.org/docs/stable/generated/torch.nn.CosineEmbeddingLoss.html

In [14]:
criterion = torch.nn.CosineEmbeddingLoss()

v1 = torch.randn((16, 64))
v2 = torch.randn((16, 64))
v2 = v1 * 0.6

target = torch.ones((v1.shape[0]))
print(target.shape)

loss = criterion(v1, v2, target)
print(loss)

torch.Size([16])
tensor(-1.1176e-08)


## Perceptual Loss

In [6]:
import lpips
loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
loss_fn_vgg = lpips.LPIPS(net='vgg') # closer to "traditional" perceptual loss, when used for optimization

import torch
img0 = torch.zeros(1,3,64,64) # image should be RGB, IMPORTANT: normalized to [-1,1]
img1 = torch.ones(1,3,64,64)
d = torch.mean(loss_fn_vgg(img0, img1))
print(d.shape)
print(d)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/haimingzhang/miniconda3/envs/GAN/lib/python3.6/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /home/haimingzhang/miniconda3/envs/GAN/lib/python3.6/site-packages/lpips/weights/v0.1/vgg.pth
torch.Size([])
tensor(0.4945, grad_fn=<MeanBackward0>)


In [4]:
import torch
import torch.nn as nn


input1 = torch.randn(50, 128, 1)
input2 = torch.randn(1, 128, 100)

cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(input1, input2)
print(output.shape)

torch.Size([50, 100])


## Total Variation Loss

In [10]:
import torch
import cv2

def total_variation_loss(img):
    bs_img, c_img, h_img, w_img = img.size()
    tv_h = torch.pow(img[:,:,1:,:]-img[:,:,:-1,:], 2).sum()
    tv_w = torch.pow(img[:,:,:,1:]-img[:,:,:,:-1], 2).sum()
    return (tv_h + tv_w) / (bs_img*c_img*h_img*w_img)

img = cv2.imread("/home/zhanghm/Research/V100/programming-learning-examples/Python_learning/data/obama.jpg")
img_pred = torch.rand(1,3,64,64)
img_pred = torch.FloatTensor(img)[None] / 255
img_pred = img_pred.permute(0,3,1,2)

loss = total_variation_loss(img_pred)
print(loss)

tensor(0.0004)
